In [ ]:
import numpy as np
import pandas as pd
import os 
import requests
import json

- ## connect Impala

In [ ]:
from impala.dbapi import connect 
conn = connect(host = '192.168.17.132', port=21050)
cursor = conn.cursor()

- ## Loading crawled file
    - ### change table to dataframe

In [ ]:
file = 'station_food_0_11'
df = pd.read_csv('/home/training/Desktop/data/{filename}_Insta_data.csv'.format(filename = file))
df.columns = ["index","date","time","site"]
df = df.iloc[1:,1:]
df = df.dropna() # delete null data
import re
for i in range(len(df.iloc[:,2])):
    df.iloc[i,2] = re.sub('[^\sa-zA-Z0-9ㄱ-ㅣ가-힣]','',df.iloc[i,2])
df.to_csv('/home/training/Desktop/new_insta_data.csv')
df

In [ ]:
df["x"] = ""
df["y"] = ""
df

- ## get unique site 

In [ ]:
site = df["site"].unique()

- ## find latitude and longitude from map search using KAKAO map API

In [ ]:

searched_site = pd.DataFrame(columns = ["site","place_name","x","y"])
for i in site:
    print(i)
    url = "https://dapi.kakao.com/v2/local/search/keyword.json?y=37.514322572335935&x=127.06283102249932&radius=20000&query={site}".format(site = i)
    headers = {"Authorization": "KakaoAK 3541027113e812edb537fe1909d43b44"}
    api_test = requests.get(url, headers = headers)
    api_test = json.loads(api_test.text)
    print(api_test)
    try:
        k = api_test["documents"]
        print("test")
        if(k):
            k = k[0]
            # 해당하는 site에 좌표 넣어주기
            df.loc[df["site"] == i, "x"] = k["x"]
            df.loc[df["site"] == i, "y"] = k["y"]
            for j in api_test["documents"]:
                searched_site = searched_site.append({"site":i, "place_name":j["place_name"], "x":j["x"], "y":j["y"]}, ignore_index = True)
    except:
        pass
searched_site

- ## add Latitude longitude to dataframe

In [ ]:
df['x'].replace('', np.nan, inplace=True)
df = df.dropna()

- ## export mid_result to local in csv format

In [ ]:
df.to_csv('/home/training/Desktop/mid_result/{filename}_insta_test_new.csv'.format(filename = file),sep = ',', columns = ['date','time','site',"x","y"],index = False,encoding = 'UTF-8-sig')

- ## input mid_result csv from local to HDFS

In [ ]:
os.system('hdfs dfs -put /home/training/Desktop/mid_result/{filename}_insta_test_new.csv /assignment1/'.format(filename = file))

- ## make a result_test

In [ ]:
cursor.execute('CREATE TABLE insta_result_test (_date string, _time string, _location string, x string, y string) row format delimited fields terminated by ","')

- ## insert overwrite test_new into result_test

In [ ]:
cursor.execute("load data inpath '/assignment1/{filename}_insta_test_new.csv' overwrite into table insta_result_test".format(filename = file))
cursor.execute('SELECT * from insta_result_test')
cursor.fetchall()

- ## Assign each post its section

In [ ]:
cursor.execute('drop table insta_filled')
cursor.execute('''create table insta_filled as
select 
    tt._location as loc,
    tt.x as x,
    tt.y as y,
    ss.id as section_id
from
    insta_result_test tt join
    section ss on
        ss.lt_lat >= cast(tt.y as float) and
      ss.lt_lat-0.001 <= cast(tt.y as float) and
      ss.lt_lng <= cast(tt.x as float) and
      ss.lt_lng+0.001 >= cast(tt.x as float)''')
cursor.execute('select * from insta_filled')
cursor.fetchall()

- ## Counting number of post for each section

In [ ]:
cursor.execute('drop table density')
cursor.execute('''create table density as
select
    section_id,
    count(*)
from
    insta_filled
group by
    section_id;''')

In [ ]:
cursor.execute('SELECT * FROM density')
result = cursor.fetchall()
result

In [ ]:
import csv
with open('/home/training/Desktop/result/{filename}insta_result.csv'.format(filename=file),'w',newline='') as f:
    write = csv.writer(f)
    
    write.writerows(result)

In [ ]:
cursor.execute('DROP TABLE insta_result_test')

- ## export result file to server MySQL

In [ ]:
os.system('scp -i /home/training/Desktop/dbwp-itm-key.pem /home/training/Desktop/result/{filename}insta_result.csv ubuntu@ec2-3-34-91-195.ap-northeast-2.compute.amazonaws.com:/home/ubuntu/ys-result'.format(filename=file))

In [ ]:
os.system('sqoop export --connect jdbc:mysql:ec2-3-34-91-195.ap-northeast-2.compute.amazonaws.com --username ubuntu --password 1234 -- table insta_result_test')